In [25]:
# Text generation example using Wordnet

In [26]:
from nltk.corpus import wordnet as wn

In [27]:
# take an example

In [28]:
# Check for different sentences
sentence = "John is a good boy."
sentence = "John runs fast."
# sentence = "The quick brown fox jumps over the lazy dog."

In [29]:
from nltk import word_tokenize
from nltk import pos_tag

In [31]:
# import nltk
# nltk.download('wordnet')
# nltk.download('all')
# wn.synonyms('car')

In [32]:
tokens = word_tokenize(sentence)
pos_tags = pos_tag(tokens)
pos_tags

[('John', 'NNP'), ('runs', 'VBZ'), ('fast', 'RB'), ('.', '.')]

In [33]:
from re import search

In [34]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
from random import choice
from random import random

In [35]:
lexical_category = ''
new_words = []
for token, pos in pos_tags:
    if search('^VB.*$', pos):
        lexical_category = 'v'
    elif search('^NN[S]?$', pos):
        lexical_category = 'n'
    elif search('^JJ$', pos):
        lexical_category = 'a'
    elif search('^RB$', pos):
        lexical_category = 'r'
    if lexical_category:
        synonyms = []
        stemmed = stemmer.stem(token)
        if not wn.synsets(stemmed):
            stemmed = token
        for syn in wn.synsets(stemmed):
            for lemma in syn.lemmas():
                if lexical_category in ['a', 'r']:
                    antonyms = []
                    # replace with antonym if it exists else replace a synonym
                    for ant in lemma.antonyms():
                        if ant.name() != stemmed.lower() and '_' not in ant.name() and ant.name() not in antonyms:
                            antonyms.append(ant.name())
                    if antonyms:
                        antonym = choice(antonyms)
                    else:
                        antonym = lemma.name()
                    synonyms.append(antonym)
                elif lexical_category not in ['a', 'r'] and lemma.name() != stemmed.lower() and '_' not in lemma.name() and lemma.name() not in synonyms:
                    synonyms.append(lemma.name())
            if synonyms:
                new_word = choice(synonyms)
            else:
                new_word = token
    else:
        new_word = token
    new_words.append(new_word)
    lexical_category = ''
print(' '.join(new_words))

John endure fast .


In [36]:
# # movie_review dataset
from nltk.corpus import movie_reviews
# Prints file ids of positive reviews
sample_pos_files = movie_reviews.fileids('pos')[: 500]
# Prints file ids of negative reviews.
sample_neg_files = movie_reviews.fileids('neg')[: 500]
all_text_reviews = ''
for sample_file in sample_pos_files + sample_neg_files:
    text = movie_reviews.raw(sample_file)
    # preprocess the data
    text = text.replace('\n', '')
    all_text_reviews += text + ' '
all_text_reviews = all_text_reviews.strip()

In [37]:
from nltk.corpus import gutenberg
# Prints file ids of gutenberg
sample_fileids = gutenberg.fileids()[: 100]
all_text_gutenberg = ''
for sample_file in sample_fileids:
    text = gutenberg.raw(sample_file)
    # preprocess the data
    text = text.replace('\n', ' ')
    all_text_gutenberg += text + ' '
all_text_gutenberg = all_text_gutenberg.strip()
print(len(all_text_gutenberg), len(all_text_gutenberg.split()))

11793333 2135242


In [38]:
from nltk.corpus import treebank
# Prints file ids of WSJ in treebank
sample_fileids = treebank.fileids()[: 1000]
all_text_treebank = ''
for sample_file in sample_fileids:
    text = treebank.raw(sample_file)
    # preprocess the data
    text = text.replace('\n', ' ')
    all_text_treebank += text + ' '
all_text_treebank = all_text_treebank.strip()

In [39]:
import nltk
from collections import defaultdict, Counter
import random

# --- 1. Sample Corpus ---
# For a real application, you would load a much larger text file.
# For this example, we'll use a small, simple corpus.
# sample_corpus = """
# Natural language processing (NLP) is a field of artificial intelligence, 
# computer science, and computational linguistics concerned with the 
# interactions between computers and human (natural) languages. 
# As such, NLP is about programming computers to process and analyze 
# large amounts of natural language data. Challenges in natural language 
# processing often involve speech recognition, natural language understanding, 
# and natural language generation.
# """
# select the corpus of your choice
corpus = all_text_gutenberg
# --- 2. Preprocessing the Corpus ---
# Tokenization: Breaking text into words/tokens
# Lowercasing: Normalizing words to avoid treating "The" and "the" as different.
# Punctuation removal (optional, but good for basic models)

def preprocess_text(text, flag='bi'):
    # Tokenize the text into words
    all_words = []
    for sentence in nltk.sent_tokenize(text):
        tokens = nltk.word_tokenize(sentence.lower())
        # Filter out non-alphabetic tokens (basic punctuation removal)
        # You might want more sophisticated handling for numbers, contractions etc.
        words = [word for word in tokens if word.isalpha()]
        if flag == 'bi':
            words = ['<S>'] + words + ['</S>']
        else:
            words = ['<S>', '<S>'] + words + ['</S>', '</S>']
        all_words += words
    return all_words

processed_words_bigrams = preprocess_text(corpus, 'bi')
print('Done')

Done


In [40]:
print(len(processed_words_bigrams), processed_words_bigrams[: 5])

2283281 ['<S>', 'emma', 'by', 'jane', 'austen']


In [41]:
def create_unigram_count_dict(words):
    """Create unigram count dict."""
    return Counter(words)

In [42]:
unigram_counts = create_unigram_count_dict(processed_words_bigrams)
print(len(unigram_counts))

40794


In [43]:
# Sentence creation using unigram model
total_unigrams = sum(unigram_counts.values())
# Using Lidstone smoothing
alpha = 0.5
vocab_size = len(unigram_counts)
unigram_prob = {word: unigram_counts[word] + alpha / total_unigrams + vocab_size for word in unigram_counts}
# random sentence generation with fixed no of words
sentence_length = 20
enumerated_dict = dict(enumerate(unigram_prob.keys()))
total_indexes = range(len(enumerated_dict))
for i in range(3):
    words = []
    for j in range(sentence_length):
        random_word = enumerated_dict[choice(total_indexes)]
        if random_word == '</S>':
            break
        else:
            words.append(random_word)
    # generates garbage
    print(' '.join(words))

pie bedtime baptised registred getting erech thronged riuer unweighed peniel went campaign collapsing hostesse uncrossed nickname ignored gaal controls qualifies
ripen mercy clipp merari cruised quietus tubercled rallying glum candor aladdin octagon arba tartar japhet learning ideals level careering belch
spent enquirest blossomes iuggel linen refraction mattithiah trick carpenter molest meekin healthier snoozing hepherites instructions timed difficulties gezrites pecking superinduced


In [44]:
# determine the vocabulary size
vocab_size = len(unigram_counts)
print(vocab_size)

40794


In [45]:
# Creating bigrams
def create_bigrams(words):
    # Add start and end tokens.
    # '<S>' for start of sentence, '</S>' for end of sentence.
    # For a simple corpus like this, we'll just treat the whole thing as one "sentence" for simplicity,
    # or you can split by actual sentences first.
    # For now, let's just use a single start/end token for the whole text for simplicity of demonstration.
    # For multiple sentences, you'd apply this per sentence after sentence tokenization.
    
    # Example for a full text with sentences:
    # sentences = nltk.sent_tokenize(text.lower())
    # all_bigrams = []
    # for sent in sentences:
    #     tokens = [word for word in nltk.word_tokenize(sent) if word.isalpha()]
    #     padded_tokens = ['<S>'] + tokens + ['</S>']
    #     for i in range(len(padded_tokens) - 1):
    #         all_bigrams.append(tuple(padded_tokens[i:i + 2]))
    # return all_bigrams

    # Simpler approach for this small text example:
    # padded_words = ['<S>'] + words + ['</S>'] # Pad with one start/end token
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append(tuple(words[i: i + 2]))
    return bigrams

bigrams = create_bigrams(processed_words_bigrams)
print('Done')

def build_bigram_model(bigrams, vocab_size=0, smoothing=False, alpha=0):
    bigram_counts = {}

    for w1, w2 in bigrams:
        bigram_counts.setdefault(w1, {})
        bigram_counts[w1][w2] = bigram_counts[w1].get(w2, 0) + 1
    # Convert counts to probabilities (smoothed with Lidstone smoothing if needed, but omitted for simplicity here)
    # For each w1 unigram, calculate P(w2 | w1)
    bigram_probabilities = {}
    for w1 in bigram_counts:
        total_unigram_count = sum(bigram_counts[w1].values())
        for w2, count in bigram_counts[w1].items():
            bigram_probabilities.setdefault(w1, {})
            if not smoothing:
                bigram_probabilities[w1][w2] = count / total_unigram_count
            else:
                bigram_probabilities[w1][w2] = count + alpha / total_unigram_count + alpha * vocab_size
            
    return bigram_probabilities, bigram_counts # Also return counts for debugging/analysis

# Probabilities with smoothing
alpha = 0.5
smoothing = True
bigram_model, bigram_counts_raw = build_bigram_model(bigrams, vocab_size, smoothing, alpha)

Done


In [46]:
from math import log
def generate_text_using_bigram_model(model, num_words=20, start_sequence=('<S>',)):
    generated_text = list(start_sequence)
    # find the log prob score
    log_prob_score = 0
    for _ in range(num_words):
        current_unigram = generated_text[-1]
        # Get possible next words and their probabilities
        possible_next_words_probs = model.get(current_unigram)
        if not possible_next_words_probs:
            # If no continuation found, stop or restart with a common bigram
            # For simplicity, we'll stop here. In a more robust model, you'd handle this.
            # print(f"\nNo known continuation for {current_bigram}. Stopping generation.")
            break
            
        # Select the next word based on probabilities
        words = list(possible_next_words_probs.keys())
        probabilities = list(possible_next_words_probs.values())
        
        next_word = random.choices(words, weights=probabilities, k=1)[0]
        log_prob_score += log(model[current_unigram][next_word])
        generated_text.append(next_word)
        
        # Stop if we generate an end-of-sentence token
        if next_word == '</S>':
            break
            
    # Clean up the generated text (remove start/end tokens, join words)
    clean_text_tokens = [word for word in generated_text if word not in ['<S>', '</S>']]
    print('Log Probability of the generated text =', log_prob_score)
    return " ".join(clean_text_tokens)

# --- 6. Generate Text ---
print(f"\n--- Generated Text ---")
for i in range(3): # Generate a few sentences
    generated_sentence = generate_text_using_bigram_model(bigram_model, num_words=20)
    print(f"Generated {i+1}: {generated_sentence.capitalize()}.") # Capitalize for better readability


--- Generated Text ---
Log Probability of the generated text = 19.846536241303905
Generated 1: Ex.
Log Probability of the generated text = 198.46588717972807
Generated 2: Jump into stillness here inquiring tireless and scarlet embrace some dosen or ours be righteousness godliness looking little brothers unasked.
Log Probability of the generated text = 29.77670959763559
Generated 3: Evidently about.


In [48]:
processed_words_trigrams = preprocess_text(corpus, 'tri')

In [51]:
# --- 3. Creating Trigrams ---
# A trigram is a sequence of three consecutive words.
# We'll also add special start/end tokens to handle sentence boundaries
# and allow generation to start and end naturally.
# Smoothing can also be incorporated
def create_trigrams(words):
    # Add start and end tokens.
    # '<S>' for start of sentence, '</S>' for end of sentence.
    # For a simple corpus like this, we'll just treat the whole thing as one "sentence" for simplicity,
    # or you can split by actual sentences first.
    # For now, let's just use a single start/end token for the whole text for simplicity of demonstration.
    # For multiple sentences, you'd apply this per sentence after sentence tokenization.
    
    # Example for a full text with sentences:
    # sentences = nltk.sent_tokenize(text.lower())
    # all_trigrams = []
    # for sent in sentences:
    #     tokens = [word for word in nltk.word_tokenize(sent) if word.isalpha()]
    #     padded_tokens = ['<S>', '<S>'] + tokens + ['</S>']
    #     for i in range(len(padded_tokens) - 2):
    #         all_trigrams.append(tuple(padded_tokens[i:i+3]))
    # return all_trigrams

    # Simpler approach for this small text example:
    # padded_words = ['<S>', '<S>'] + words + ['</S>', '</S>'] # Pad with two start/end tokens
    trigrams = []
    for i in range(len(words) - 2):
        trigrams.append(tuple(words[i: i + 3]))
    return trigrams

trigrams = create_trigrams(processed_words_trigrams)
# print(f"\nSample Trigrams (first 5): {trigrams[:5]}")
# print(f"Sample Trigrams (last 5): {trigrams[-5:]}")


# --- 4. Building the Trigram Probability Model ---
# We need:
# 1. Counts of (word1, word2, word3) - trigram_counts
# 2. Counts of (word1, word2) - bigram_counts (for the denominator)

def build_trigram_model(trigrams, vocab_size=0, smoothing=False, alpha=0):
    trigram_counts = {}

    for w1, w2, w3 in trigrams:
        trigram_counts.setdefault((w1, w2), {})
        trigram_counts[(w1, w2)][w3] = trigram_counts[(w1, w2)].get(w3, 0) + 1
    # return trigram_counts, bigram_counts
    # Convert counts to probabilities (smoothed with Lidstone smoothing if needed, but omitted for simplicity here)
    # For each (w1, w2) bigram, calculate P(w3 | w1, w2)
    trigram_probabilities = {}
    for w1, w2 in trigram_counts:
        total_bigram_count = sum(trigram_counts[(w1, w2)].values())
        for w3, count in trigram_counts[(w1, w2)].items():
            trigram_probabilities.setdefault((w1, w2), {})
            trigram_probabilities[(w1, w2)][w3] = count + alpha / total_bigram_count + alpha * vocab_size
            
    return trigram_probabilities, trigram_counts # Also return counts for debugging/analysis

# Probabilities with smoothing
alpha = 0.5
smoothing = True
trigram_model, trigram_counts_raw = build_trigram_model(trigrams, vocab_size, smoothing, alpha)

# Example: Probability of 'processing' after ('language', 'natural')
# print(f"\nProbability of 'processing' after ('language', 'natural'): {trigram_model[('language', 'natural')].get('processing', 0)}")
# print(f"Count of ('language', 'natural', 'processing'): {trigram_counts_raw[('language', 'natural')]['processing']}")
# print(f"Sum of all words after ('language', 'natural'): {sum(trigram_counts_raw[('language', 'natural')].values())}")


# --- 5. Text Generation Function ---
# Start with '<S>', '<S>' and iteratively predict the next word.

def generate_text_using_trigram_model(model, num_words=20, start_sequence=('<S>', '<S>')):
    generated_text = list(start_sequence)
    
    for _ in range(num_words):
        current_bigram = tuple(generated_text[-2:])
        log_prob_score = 0
        # Get possible next words and their probabilities
        possible_next_words_probs = model.get(current_bigram)
        
        if not possible_next_words_probs:
            # If no continuation found, stop or restart with a common bigram
            # For simplicity, we'll stop here. In a more robust model, you'd handle this.
            # print(f"\nNo known continuation for {current_bigram}. Stopping generation.")
            break
            
        # Select the next word based on probabilities
        words = list(possible_next_words_probs.keys())
        probabilities = list(possible_next_words_probs.values())
        
        next_word = random.choices(words, weights=probabilities, k=1)[0]
        generated_text.append(next_word)
        log_prob_score += log(model[current_bigram][next_word])
        # Stop if we generate an end-of-sentence token
        if next_word == '</S>':
            break
            
    # Clean up the generated text (remove start/end tokens, join words)
    clean_text_tokens = [word for word in generated_text if word not in ['<S>', '</S>']]
    print('Log Probability of the generated text =', log_prob_score)
    return " ".join(clean_text_tokens)

# --- 6. Generate Text ---
print(f"\n--- Generated Text ---")
for i in range(5): # Generate a few sentences
    generated_sentence = generate_text_using_trigram_model(trigram_model, num_words=20)
    print(f"Generated {i + 1}: {generated_sentence.capitalize()}.") # Capitalize for better readability

import numpy as np
def generate_text_using_trigram_model_with_most_probable_next_word(model, num_words=20, start_sequence=('<S>', '<S>')):
    generated_text = list(start_sequence)
    
    for _ in range(num_words):
        current_bigram = tuple(generated_text[-2:])
        log_prob_score = 0
        # Get possible next words and their probabilities
        possible_next_words_probs = model.get(current_bigram)
        
        if not possible_next_words_probs:
            # If no continuation found, stop or restart with a common bigram
            # For simplicity, we'll stop here. In a more robust model, you'd handle this.
            # print(f"\nNo known continuation for {current_bigram}. Stopping generation.")
            break
            
        # Select the next word based on probabilities
        words = list(possible_next_words_probs.keys())
        probabilities = list(possible_next_words_probs.values())
        max_prob_index = np.argmax(np.array(probabilities))
        next_word = words[max_prob_index]
        generated_text.append(next_word)
        log_prob_score += log(model[current_bigram][next_word])
        # Stop if we generate an end-of-sentence token
        if next_word == '</S>':
            break
            
    # Clean up the generated text (remove start/end tokens, join words)
    clean_text_tokens = [word for word in generated_text if word not in ['<S>', '</S>']]
    print('Log Probability of the generated text =', log_prob_score)
    return " ".join(clean_text_tokens)

# --- 6. Generate Text with greedy method---
print(f"\n--- Generated Text With Most Probable Next Word---")
# In this greedy method you will get the same sentence every time, so the above random sampling or beam search is preferred
for i in range(5): # Generate a few sentences
    generated_sentence = generate_text_using_trigram_model_with_most_probable_next_word(trigram_model, num_words=20)
    print(f"Generated {i + 1}: {generated_sentence.capitalize()}.") # Capitalize for better readability


--- Generated Text ---
Log Probability of the generated text = 9.923249329324735
Generated 1: Decius.
Log Probability of the generated text = 9.923216647717123
Generated 2: Zoned by those faggots.
Log Probability of the generated text = 9.923216647717123
Generated 3: Book xvii.
Log Probability of the generated text = 9.923486499734262
Generated 4: Revenge at first as other people the difference that the willoughbys nothing of marianne till dinner time she knew not.
Log Probability of the generated text = 9.923216647717123
Generated 5: Larboard boat there save for civilities till they shine yea they are vain for the best outline pictures are in.

--- Generated Text With Most Probable Next Word---
Log Probability of the generated text = 9.957549513979904
Generated 1: And the lord.
Log Probability of the generated text = 9.957549513979904
Generated 2: And the lord.
Log Probability of the generated text = 9.957549513979904
Generated 3: And the lord.
Log Probability of the generated text =

In [86]:
# --- 7. Text Generation Function with beam search---
# Start with '<S>', '<S>' and iteratively predict the next word.
from random import sample
def generate_text_with_beam_search(model, num_words=20, start_sequence=('<S>', '<S>'), beam_size=1):
    # generated_text = list(start_sequence)
    beam = [(0., list(start_sequence))]
    for _ in range(num_words):
        candidates = []
        # Early stopping if all sequences have generated </S>
        if all(seq[-1] == '</S>' for _, seq in beam):
            break
        for log_prob, seq in beam:
            current_seq = seq[-2:]
            # if the last token of the current sequence is </S>, then don't expand it
            if current_seq[-1] == '</S>':
                continue
            # Get possible next words and their probabilities
            possible_next_words_probs = model.get(tuple(current_seq))
            if not possible_next_words_probs:
                # If no continuation found, stop or restart with a common bigram
                # For simplicity, we'll stop here. In a more robust model, you'd handle this.
                # print(f"\nNo known continuation for {current_bigram}. Stopping generation.")
                break
                
            # Select all the candidates and evaluate their log probs
            words = list(possible_next_words_probs.keys())
            probabilities = list(possible_next_words_probs.values())
            for word, prob in zip(words, probabilities):
                new_log_prob = log_prob + log(prob)
                new_seq = seq + [word]
                # Stop if we generate an end-of-sentence token
                candidates.append((new_log_prob, new_seq))
            candidates.sort(key=lambda x: x[0], reverse=True)
            # For the 1st word sample all words, not the best probability ones
            if _ == 0:
                beam = sample(candidates, beam_size)
            else:
                beam = candidates[: beam_size]
            # beam = candidates[: beam_size]
            candidates = []
    best_sequence = max(beam, key=lambda x: x[0])[1]
    best_sequence = [word for word in best_sequence if word not in ['<S>', '</S>']]
    return " ".join(best_sequence)

# --- 7. Generate Text with Beam Search---
print(f"\n--- Generated Text ---")
# beam_size=1 is same as greedy search as seen above
for i in range(5): # Generate a few sentences
    generated_sentence = generate_text_with_beam_search(trigram_model, num_words=20, beam_size=5)
    print(f"Generated {i + 1}: {generated_sentence.capitalize()}.") # Capitalize for better readability



--- Generated Text ---
Generated 1: Responsive to our father which art a man in whom we think she will think this that is a very.
Generated 2: Urge and spur of every man in whom we think she will think this that is a thing that is.
Generated 3: Tiny green leaves of her being settled there had of him as she did so with some anxiety that there.
Generated 4: Italian music behind me sank back and saw and listened as animals listen for any change for heaven to give.
Generated 5: Disguised as one can only recollect the fate of mother ocean it had never thought edward so i did not.
